In [2]:
import pandas as pd
import sqlite3

import sys
import os
# Modify path temporarily to import functions from NASEM_functions directory
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

from NASEM_functions.ration_balancing_equations import *

In [19]:
# User will type the ration into input.txt
# This function reads that file and creates a dataframe with all of the feeds and % DM 

def read_input(input):
    data = []
    animal_input = {}

    with open(input, 'r') as file:
        for line in file:
            line = line.strip()

            if line.startswith('#') or ':' not in line:
                continue

            if line.startswith('*'):
                key, value = line[1:].split(":")
                animal_input[key.strip()] = float(value.strip())
                continue

            feedstuff, per_DM = line.split(":")
            data.append([feedstuff.strip(), per_DM.strip()])

    user_input = pd.DataFrame(data, columns=["Feedstuff", "%_DM_user"])
    user_input['%_DM_user'] = user_input['%_DM_user'].astype(float)
    
    user_input['Feedstuff'] = user_input['Feedstuff'].str.strip()
    # user_input.set_index('Feedstuff', inplace=True)  # Set 'Feedstuff' as the index


    user_input['Index'] = user_input['Feedstuff']
    user_input = user_input.set_index('Index') 
    # user_input.index = user_input.index.str.strip()

    # user_input = user_input.set_index('Feedstuff')
    # user_input.set_index('Feedstuff', inplace=True)
    # user_input.index = user_input.index.str.strip()
    
    return user_input, animal_input

user_input, animal_input = read_input('input.txt')
list_of_feeds = user_input['Feedstuff'].tolist()
# list_of_feeds = user_input.index.tolist()

In [20]:
# Takes a list of the feed names and gets all their info from the feed library

def fl_get_rows(feeds_to_get):
    conn = sqlite3.connect('../../diet_database.db')
    cursor = conn.cursor()

    index_str = ', '.join([f"'{idx}'" for idx in feeds_to_get])

    query = f"SELECT * FROM feed_library WHERE Name IN ({index_str})"

    cursor.execute(query)
    rows = cursor.fetchall()

    cursor.execute(f"PRAGMA table_info(feed_library)")
    column_names = [column[1] for column in cursor.fetchall()]

    # Create a DataFrame from the retrieved rows with column names
    feed_data = pd.DataFrame(rows, columns=column_names)
    feed_data = feed_data.set_index('Name')
    feed_data.index = feed_data.index.str.strip()

    conn.close()

    return feed_data

feed_data = fl_get_rows(list_of_feeds)

In [21]:
# Get %DM to add to 100%
def set_perc_100():
    user_perc = user_input['%_DM_user'].sum()
    scaling_factor = 100 / user_perc

    user_input['%_DM_intake'] = user_input['%_DM_user'] * scaling_factor

    # Adjust so sum is exactly 100
    adjustment = 100 - user_input['%_DM_intake'].sum()
    user_input['%_DM_intake'] += adjustment / len(user_input)

    return user_input

user_input = set_perc_100()

In [22]:
# Predict DMI

def dmi_predicted(An_Parity_rl, Trg_MilkProd, An_BW, An_BCS, An_LactDay, Trg_MilkFatp, Trg_MilkTPp, Trg_MilkLacp):
    DMI = calculate_Dt_DMIn_Lact1(An_Parity_rl, Trg_MilkProd, An_BW, An_BCS, An_LactDay, Trg_MilkFatp, Trg_MilkTPp, Trg_MilkLacp)
    return DMI

animal_input['DMI'] = dmi_predicted(**animal_input)


In [23]:
# Get kg intake of each feed

def get_kg_intake(df, dict):
    DMI = dict['DMI']
    df['kg_intake'] = df['%_DM_intake']/100 * DMI

get_kg_intake(user_input, animal_input)

In [8]:
# Calculate Nutrient Intakes

# I tried giving a list of columns and joining the feed data df to the user input df but there are a few issues
# 1. It is going to pull the entire column for each item in the list so it would still need to index and select the correct rows
# 2. The names in the database would need to change to not % as this is a special character in SQL, this makes it difficult to pass
# a list to the formatted text statements I've been using to interact with the database


def get_nutrient_intakes(df):
    for feed in df['Feedstuff']:
        
        # CP Intake
        df.loc[df['Feedstuff'] == feed, 'CP_%_diet'] = feed_data.loc[feed, 'CP (%DM)'] * df.loc[df['Feedstuff'] == feed, '%_DM_intake'] / 100
        df.loc[df['Feedstuff'] == feed, 'CP_kg/d_diet'] = feed_data.loc[feed, 'CP (%DM)'] * df.loc[df['Feedstuff'] == feed, 'kg_intake'] / animal_input['DMI']

        # RUP Intake
        df.loc[df['Feedstuff'] == feed, 'CP_%_diet'] = feed_data.loc[feed, 'CP (%DM)'] * df.loc[df['Feedstuff'] == feed, '%_DM_intake'] / 100


        # print(feed_data.loc[feed, 'CP (%DM)'] * 2)

get_nutrient_intakes(user_input)

In [68]:
feed = 'alfalfa meal'

print(feed_data.loc[feed, 'CP (%DM)'])

print(user_input.loc[feed, '%_DM_intake'])

print(feed_data.index)

19.2
33.48214285714286
Index(['alfalfa meal', 'canola meal', 'corn grain high moisture',
       'corn silage 30% grain'],
      dtype='object', name='Name')


In [18]:
# Calculate Nutrient Intakes

fl_columns = ['CP (%DM)', 'NDF (% DM)']

def fl_get_table(df, columns):
    
    conn = sqlite3.connect('../../diet_database.db')

    # # If there is no list given or the list is empty select the entire table
    # if columns is None or len(columns) == 0:
    #     query = f"SELECT * FROM feed_library"
    # else:
    cols_str = ', '.join(columns)
    # query = f"SELECT {cols_str} FROM feed_library"
    query = "SELECT `CP (%DM)`, `NDF (% DM)` FROM feed_library"

    
    feed_df = pd.read_sql_query(query, conn)
    # df = df.join(feed_df)
    
    conn.close()
    # return df
    return feed_df

# user_input = fl_get_table(user_input, fl_columns)
feed_df = fl_get_table(user_input, fl_columns)

# def get_nutrient_intakes(df):
    